In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from dateutil.parser import parse
%matplotlib inline
sns.set_style('white')
import datetime


In [67]:
data = pd.read_csv("predict_data.csv")
data.head()

,market_id,created_at,store_id,store_primary_category,order_protocol,total_items,subtotal,num_distinct_items,min_item_price,max_item_price,total_onshift_dashers,total_busy_dashers,total_outstanding_orders,estimated_order_place_duration,estimated_store_to_consumer_driving_duration,delivery_id,platform
0,3.0,2015-02-25 02:22:30,5477,NaN,1.0,5,7500,4,800,1800,4.0,4.0,4.0,446,670.0,194096,android
1,3.0,2015-02-25 01:14:19,5477,NaN,1.0,5,7100,4,800,1500,4.0,1.0,1.0,446,446.0,236895,other
2,4.0,2015-02-22 02:27:44,5477,thai,1.0,4,4500,2,750,1500,9.0,7.0,6.0,446,504.0,190868,android
3,3.0,2015-02-25 01:32:39,5477,NaN,1.0,1,1700,1,1400,1400,3.0,3.0,3.0,446,687.0,183076,ios
4,1.0,2015-02-24 05:20:45,2841,italian,1.0,2,3150,2,1525,1625,4.0,4.0,4.0,446,528.0,186200,android


In [51]:
data.describe()

,market_id,store_id,order_protocol,total_items,subtotal,num_distinct_items,min_item_price,max_item_price,total_onshift_dashers,total_busy_dashers,total_outstanding_orders,estimated_order_place_duration,estimated_store_to_consumer_driving_duration,delivery_id
count,54528.000000,54778.000000,54495.000000,54778.000000,54778.000000,54778.000000,54778.000000,54778.000000,50145.000000,50145.000000,50145.000000,54778.000000,54767.000000,54778.000000
mean,3.031360,3489.640969,2.885017,3.227737,2715.758772,2.691902,683.757384,1166.795356,49.076179,43.515206,62.566756,308.112673,547.765406,126118.103454
std,1.531606,2050.221801,1.512087,2.557472,1897.919808,1.650204,538.496995,574.594726,37.594714,34.051339,56.378953,88.937888,220.459715,72683.611935
min,1.000000,2.000000,1.000000,1.000000,0.000000,1.000000,-63.000000,0.000000,-1.000000,-4.000000,-9.000000,0.000000,0.000000,20.000000
25%,2.000000,1661.000000,1.000000,2.000000,1420.000000,2.000000,299.000000,819.000000,19.000000,16.000000,19.000000,251.000000,385.000000,62938.500000
50%,3.000000,3518.000000,3.000000,3.000000,2220.000000,2.000000,595.000000,1095.000000,41.000000,36.000000,45.000000,251.000000,544.000000,126194.000000
75%,4.000000,5247.000000,4.000000,4.000000,3415.000000,3.000000,929.000000,1399.000000,71.000000,63.000000,91.000000,446.000000,703.000000,188916.000000
max,6.000000,6986.000000,7.000000,81.000000,29925.000000,19.000000,25040.000000,25142.000000,175.000000,169.000000,292.000000,1095.000000,1550.000000,252031.000000


In [78]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54778 entries, 0 to 54777
Data columns (total 19 columns):
market_id                                       54778 non-null float64
created_at                                      54778 non-null datetime64[ns, US/Pacific]
store_id                                        54778 non-null int64
store_primary_category                          54778 non-null int64
order_protocol                                  54778 non-null float64
total_items                                     54778 non-null int64
subtotal                                        54778 non-null int64
num_distinct_items                              54778 non-null int64
min_item_price                                  54778 non-null int64
max_item_price                                  54778 non-null int64
total_onshift_dashers                           50145 non-null float64
total_busy_dashers                              50145 non-null float64
total_outstanding_orders          

In [69]:
from pytz import timezone
import pytz
data["created_at"] = data["created_at"].apply(lambda x:  pytz.timezone('UTC').localize(datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S")) if not pd.isna(x) else float('nan'))
data["created_at"] = data["created_at"].apply(lambda x : x.astimezone(timezone('US/Pacific')) if not pd.isna(x) else float('nan'))

data.head()

,market_id,created_at,store_id,store_primary_category,order_protocol,total_items,subtotal,num_distinct_items,min_item_price,max_item_price,total_onshift_dashers,total_busy_dashers,total_outstanding_orders,estimated_order_place_duration,estimated_store_to_consumer_driving_duration,delivery_id,platform
0,3.0,2015-02-24 18:22:30-08:00,5477,NaN,1.0,5,7500,4,800,1800,4.0,4.0,4.0,446,670.0,194096,android
1,3.0,2015-02-24 17:14:19-08:00,5477,NaN,1.0,5,7100,4,800,1500,4.0,1.0,1.0,446,446.0,236895,other
2,4.0,2015-02-21 18:27:44-08:00,5477,thai,1.0,4,4500,2,750,1500,9.0,7.0,6.0,446,504.0,190868,android
3,3.0,2015-02-24 17:32:39-08:00,5477,NaN,1.0,1,1700,1,1400,1400,3.0,3.0,3.0,446,687.0,183076,ios
4,1.0,2015-02-23 21:20:45-08:00,2841,italian,1.0,2,3150,2,1525,1625,4.0,4.0,4.0,446,528.0,186200,android


In [70]:
import holidays
data["day_week"] = pd.DatetimeIndex(data["created_at"]).dayofweek
data["day"] = pd.DatetimeIndex(data["created_at"]).day
#data["month"] = pd.DatetimeIndex(data["actual_delivery_time"]).month
data["hour_created"] = pd.DatetimeIndex(data["created_at"]).hour
#data["holiday"] = data["created_at"].apply(lambda x : 1 if x in  holidays.UnitedStates() else 0)


In [71]:
data.drop(["platform"],axis=1,inplace=True)

In [72]:
from sklearn import preprocessing
# label_encoder object knows how to understand word labels. 
label_encoder = preprocessing.LabelEncoder()
# Encode labels in column 'Country'. 
data['store_primary_category'].fillna("Unknown", inplace = True) 
data['store_primary_category']= label_encoder.fit_transform(data['store_primary_category']) 

In [73]:
from collections import defaultdict
store_dict = defaultdict(list)
market_missing = data.groupby(['store_id',"market_id"])["created_at"].count().reset_index().sort_values(by=['store_id'])
for i,row in market_missing.iterrows():
    store_dict[row["store_id"]].append((row["market_id"],row["created_at"]))


#### Handling missing values for market_id using most common store_id in given market_id

In [74]:
def fillmarketid(row):
    if row["market_id"] == "None":
        tuples = (sorted(store_dict[row["store_id"]],key=lambda elem: elem[1]))
        if len(tuples) > 0 : row["market_id"] = tuples[0][0] 
        else : row["market_id"] = 2.0    
    return row

In [75]:
data["market_id"].fillna("None",inplace=True) 
data = data.apply(lambda x : fillmarketid(x),axis=1 )

#### handling missing values for order_protocol with 1 and 3 using estimated_order_place duration value

In [76]:
def fillorderprotocol(row):
    if row["order_protocol"] == "None":
        if row["estimated_order_place_duration"] < 300 :
            row["order_protocol"] = 3.0
        else : row["order_protocol"] = 1.0    
    return row

In [77]:
data.order_protocol.fillna("None",inplace=True)
data = data.apply(lambda x: fillorderprotocol(x),axis=1)

In [79]:
timeReqperCat = pd.read_csv("C:/Users/Prajakta Gujarathi/Documents/Python Scripts/Untitled Folder/delivery_estimate/timeReqperCat.csv")
timeReqperstorhour = pd.read_csv("C:/Users/Prajakta Gujarathi/Documents/Python Scripts/Untitled Folder/delivery_estimate/timeReqperstorhour.csv")
data = data.merge(timeReqperstorhour,how="left", on=["store_id", "hour_created"])
data = data.merge(timeReqperCat, how = "left",on=["store_primary_category","hour_created"])
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 54778 entries, 0 to 54777
Data columns (total 21 columns):
market_id                                       54778 non-null float64
created_at                                      54778 non-null datetime64[ns, US/Pacific]
store_id                                        54778 non-null int64
store_primary_category                          54778 non-null int64
order_protocol                                  54778 non-null float64
total_items                                     54778 non-null int64
subtotal                                        54778 non-null int64
num_distinct_items                              54778 non-null int64
min_item_price                                  54778 non-null int64
max_item_price                                  54778 non-null int64
total_onshift_dashers                           50145 non-null float64
total_busy_dashers                              50145 non-null float64
total_outstanding_orders          

In [80]:
data.drop(["created_at"],axis=1,inplace=True)

In [81]:
from impyute.imputation.cs import mice

# start the MICE training
imputed_training=mice(data.values)
df = pd.DataFrame(imputed_training)
df.columns = data.columns
df.describe()

,market_id,store_id,store_primary_category,order_protocol,total_items,subtotal,num_distinct_items,min_item_price,max_item_price,total_onshift_dashers,total_busy_dashers,total_outstanding_orders,estimated_order_place_duration,estimated_store_to_consumer_driving_duration,delivery_id,day_week,day,hour_created,avg_time_to_prepare_store_hour,avg_time_to_prepare_cat_hour
count,54778.000000,54778.000000,54778.000000,54778.000000,54778.000000,54778.000000,54778.000000,54778.000000,54778.000000,54778.000000,54778.000000,54778.000000,54778.000000,54778.000000,54778.000000,54778.000000,54778.000000,54778.000000,54778.000000,54778.000000
mean,3.030815,3489.640969,34.783471,2.882215,3.227737,2715.758772,2.691902,683.757384,1166.795356,49.095406,43.446635,62.956797,308.112673,547.766144,126118.103454,3.292983,20.961901,16.054036,1899.169541,1945.746590
std,1.532090,2050.221801,20.648764,1.510187,2.557472,1897.919808,1.650204,538.496995,574.594726,35.971178,32.580656,53.967518,88.937888,220.437664,72683.611935,1.986633,1.874202,3.291560,527.195695,427.499107
min,1.000000,2.000000,0.000000,1.000000,1.000000,0.000000,1.000000,-63.000000,0.000000,-1.000000,-4.000000,-9.000000,0.000000,0.000000,20.000000,0.000000,18.000000,6.000000,-261.000000,512.000000
25%,2.000000,1661.000000,15.000000,1.000000,2.000000,1420.000000,2.000000,299.000000,819.000000,20.000000,18.000000,21.000000,251.000000,385.000000,62938.500000,2.000000,19.000000,13.000000,1553.393939,1703.534091
50%,3.000000,3518.000000,38.000000,3.000000,3.000000,2220.000000,2.000000,595.000000,1095.000000,46.000000,41.000000,53.000000,251.000000,544.000000,126194.000000,4.000000,21.000000,17.000000,1855.000000,1915.580645
75%,4.000000,5247.000000,54.000000,4.000000,4.000000,3415.000000,3.000000,929.000000,1399.000000,67.000000,60.000000,85.000000,446.000000,703.000000,188916.000000,5.000000,22.000000,18.000000,2215.875000,2161.500000
max,6.000000,6986.000000,71.000000,7.000000,81.000000,29925.000000,19.000000,25040.000000,25142.000000,175.000000,169.000000,292.000000,1095.000000,1550.000000,252031.000000,6.000000,24.000000,23.000000,4506.000000,4598.000000


#### converting all negative value to positive

In [82]:
df = df.abs()

In [86]:
df = df.set_index("delivery_id")
df.head()

,market_id,store_id,store_primary_category,order_protocol,total_items,subtotal,num_distinct_items,min_item_price,max_item_price,total_onshift_dashers,total_busy_dashers,total_outstanding_orders,estimated_order_place_duration,estimated_store_to_consumer_driving_duration,day_week,day,hour_created,avg_time_to_prepare_store_hour,avg_time_to_prepare_cat_hour
delivery_id,,,,,,,,,,,,,,,,,,,
194096.0,3.0,5477.0,0.0,1.0,5.0,7500.0,4.0,800.0,1800.0,4.0,4.0,4.0,446.0,670.0,1.0,24.0,18.0,897.000000,2327.951460
236895.0,3.0,5477.0,0.0,1.0,5.0,7100.0,4.0,800.0,1500.0,4.0,1.0,1.0,446.0,446.0,1.0,24.0,17.0,1616.333333,1901.976672
190868.0,4.0,5477.0,67.0,1.0,4.0,4500.0,2.0,750.0,1500.0,9.0,7.0,6.0,446.0,504.0,5.0,21.0,18.0,897.000000,2362.772340
183076.0,3.0,5477.0,0.0,1.0,1.0,1700.0,1.0,1400.0,1400.0,3.0,3.0,3.0,446.0,687.0,1.0,24.0,17.0,1616.333333,1901.976672
186200.0,1.0,2841.0,38.0,1.0,2.0,3150.0,2.0,1525.0,1625.0,4.0,4.0,4.0,446.0,528.0,0.0,23.0,21.0,2158.387115,1942.379976


In [88]:
import pickle
model = pickle.load(
           open("C:/Users/Prajakta Gujarathi/Documents/Python Scripts/Untitled Folder/total_delivery_estimate.sav",
               "rb"))

In [ ]:
df["predicted_duration"] = model.predict(df.drop(["avg_time_to_prepare_cat_hour","store_id","store_primary_category","total_items"],axis=1))
df.reset_index(inplace=True)
df[["delivery_id","predicted_duration"]].to_csv("Prediction.csv")